**Data Wrangling**


**PCA Code**

In [4]:
# import packages
import numpy as np
import scipy.io as sio
import matplotlib
import matplotlib.pyplot as plt
from numpy.matlib import repmat
from sklearn.preprocessing import normalize

The following code wrangles the images, turning them into arrays, and loads them in as a dataset, simpsons_df, so that we can work with it in our project.

In [7]:
# import dataset

# import additional packages to process images as dataset
import os
from PIL import Image

# wrangle images and transform into arrays
def load_images_from_folder(folder_path, target_size=(200, 200)):
    images = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            # Load image
            img = Image.open(file_path).convert('L')  # Convert to grayscale
            img = img.resize(target_size)  # Resize to target size
            images.append(np.array(img).flatten())  # Flatten to 1D array
        except Exception as e:
            print(f"Error loading image {filename}: {e}")
    return np.array(images)

# Load dataset
folder_path = "simpsons_faces_kostastokis/cropped"
simpsons_df = load_images_from_folder(folder_path)

# Check the shape of the dataset
print(f"Dataset shape: {simpsons_df.shape}")



Dataset shape: (9877, 40000)


In [8]:

# convert images into a 1D array
flattened_images = images.reshape(image.shape[0], -1)

# normalize the pixel values,
#      to have a range of [0, 1],
#      standardize to have mean of 0 and s.d. of 1
scaler = StandardScaler()
images_normalized = scaler.fit_transform(flattened_images)


KeyboardInterrupt: 

In [14]:
def viewcolumn(columnvector):
    # viewcolumn(columnvector);
    # VIEWCOLUMN Displays a 60 x 60 grayscale image stored in a column vector.
    # Tim Marks 2002
    plt.imshow(columnvector.reshape([60, 60], order='F'), cmap=plt.get_cmap('gray'))


def normc(Mat):
    # normc(M) normalizes the columns of M to a length of 1.
    return normalize(Mat, norm='l2', axis=0)

Compute mean face:

In [13]:
mean_simp = np.mean(simpsons_df, axis=1)

# View a transformed version of mean

mean_simp = mean_simp[:, np.newaxis]
viewcolumn(mean_simp)

ValueError: cannot reshape array of size 9877 into shape (200,200)

**PCA Code for Each Class**

**Test: Reconstruct each PCA model, then measure distance between reconstruction and input**

**Find & Discuss Lowest Distance**